In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/2017_food_posoning_regression/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/2017_food_posoning_regression


In [ ]:
!pip install flask_ngrok

In [ ]:
import json, math
import numpy as np
from pickle import load
from tensorflow.keras.models import load_model

from flask import Flask, request, Response, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
label_list = ['병원성 대장균', '살모넬라', '노로바이러스', '캠필로박터제주니', '클로스트리디움퍼프린젠스']
loaded_model = load_model('./best_model.h5')
loaded_scaler = load(open('./robust_scaler.pkl', 'rb'))

In [ ]:
def get_predict_result(x_test):
  x_test = np.reshape(x_test, (-1, 6))
  x_test_scaled = loaded_scaler.transform(x_test)
  return loaded_model.predict(x_test_scaled).tolist()

def get_percent_predict(predict):
  result =[]
  for item in predict:
    result.append(round(item * 100, 2))
  return result

def get_response(predict):
  response = {}
  response['floatList'] = predict
  return response

In [ ]:
label_list = ['병원성 대장균', '살모넬라', '노로바이러스', '캠필로박터제주니', '클로스트리디움퍼프린젠스']
loaded_model = load_model('./best_model.h5')
loaded_scaler = load(open('./robust_scaler.pkl', 'rb'))

# 2022.06.15 오후 3시 35분 기후 데이터
weather_data = [24, 14, 24, 69, 2, 4]
result = get_percent_predict(get_predict_result(weather_data)[0])

for item in zip(label_list, result):
  print("{}의 발생 확률은 {}%입니다.".format(item[0], item[1]))

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
  return "hello flask"

@app.route('/model-predict', methods=['POST'])
def model_predict_response():
  data = request.json
  x_data = data['data']
  predict = get_percent_predict(get_predict_result(x_data)[0])
  predict[0], predict[1] = predict[1], predict[0]
  print(x_data, predict)
  return jsonify(get_response(predict))

In [ ]:
run_with_ngrok(app)
app.run()